## 0. Introduction

**结论**

1. 实现了基于离散属性和连续属性的朴素贝叶斯分类器
2. 进行了二项检验、t检验、交叉验证t检验、McNemar检验，得出以下结论：
    1. 通过二项检验发现，对于离散属性的朴素贝叶斯分类器，在**99%的置信水平**下，认为模型分**泛化错误率小于42%**
    2. 通过t检验发现，对于离散属性的朴素贝叶斯分类器，在以**90%的置信度**下，认为模型的**泛化错误率等于8%**
    3. 通过交叉验证t检验发现，离散属性的朴素贝叶斯分类器的性能比连续属性的更优（**未考虑到独立采样**）
    4. 通过$5\times 2$折交叉验证t检验发现，两者性能**没有显著差别**
    5. 通过McNemar检验发现，两者性能**没有显著差别**
    

**数据**

    - 2021冬模式识别数据收集.xlsx
   
> 说明:
>
> 以身高、体重、脚长、尺码来判断性别
>
> 以上特征可以视为离散型属性，亦可以视为连续型属性
 
 **优点**：小样本，多分类

**缺点**：对于输入数据敏感

**公式**

$$
P(c|x)=\frac{p(c,x)}{p(x)}=\frac{P(c)p(x|c)}{p(x)}=\frac{P(c)}{p(x)}\prod_{i=1}^{d}{P(x_i|c)}
$$

对于所有类别来说，$P(x)$一样，因此只考虑分子部分

$$
h_{nb}=arg\ max_{c\in \gamma}P(c)\prod_{i=1}^d{P(x_i|c)}
$$

1. 离散
$$
p(c)=\frac{|D_c|}{|D|}
$$

> 拉普拉斯修正 $p(c)=\frac{|D_c|+1}{|D|+N}$


2. 连续

> 假设$p(x_i|c)\sim N(\mu_{c,i},\sigma_{c,i}^2)$

$$
p(x_i|c)=\frac{1}{\sqrt{2\pi}\sigma_{c,i}} exp(-\frac{(x_i-\mu_{c,i})^2}{2\sigma_{c,i}^2})
$$
 
 **环境**
 
 ```shell
 Python                         3.8.4
 matplotlib                     3.5.2
 numpy                          1.22.4
 pandas                         1.3.5
 scikit-learn                   1.1.1
 seaborn                        0.11.2
 ```

## 1. 导入包和模块

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split, KFold
from collections import Counter

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False   # 解决保存图像是负号'-'显示为方块的问题

warnings.filterwarnings('ignore')

In [2]:
def load_data(fileName):
    data = pd.read_excel(fileName,index_col=0)
    data = data.dropna(axis=1)
    data['性别'] = data['性别'].map({'男':0,'女':1})
    # sns.pairplot(data)
    return data

## 2. 离散属性的朴素贝叶斯分类器

In [3]:
# 定义朴素贝叶斯分类器-离散型
class NaiveBayesClassifierDiscrete:
    def __init__(self):
        self.data = None
        self.labels = None
        self.prior_prob = None
        self.conditional_prob = None
        self.num_values = None

    # 训练函数
    def fit(self, data, labels):
        """
        函数说明:
            - 计算先验概率和条件概率        
        parameters:
            - data 训练特征矩阵
            - labels 训练标签向量
        return:
            - self
        """
        self.data = data
        self.labels = labels
        # 计算先验概率
        label_counts = Counter(labels)  # 计算每个标签的数量
        num_samples = len(labels)  # 计算样本总数
        self.prior_prob = {label: count / num_samples for label, count in label_counts.items()}  # 计算每个标签的先验概率

        # 计算条件概率
        self.conditional_prob = {}
        self.num_values = {}
        for label in label_counts.keys():
            self.conditional_prob[label] = {}
            for feature_index in range(len(data[0])):
                self.conditional_prob[label][feature_index] = {}
                label_indexes = np.where(labels == label)[0]
                feature_values = data[label_indexes, feature_index]
                value_counts = Counter(feature_values)  # 统计每个特征值出现的次数
                self.num_values[feature_index] = len(feature_values)  # 该特征值出现的总次数
                # self.conditional_prob[label][feature_index] = {value: count / num_values for value, count in value_counts.items()}  # 计算每个特征值的条件概率
                # 拉普拉斯变换修正
                self.conditional_prob[label][feature_index] = {value: (count+1) / (2 * self.num_values[feature_index]) for value, count in value_counts.items()}  # 计算每个特征值的条件概率
        

    # 预测函数
    def predict(self, data):
        """
        函数说明:
            - 给定特征值，预测类别
        parameters:
            - data 测试特征向量
        return:
            - self
        """
        labels = []
        for sample in data:
            # 计算每个标签的概率
            label_prob = {label: self.prior_prob[label] for label in self.prior_prob.keys()}
            for label in self.prior_prob.keys():
                for feature_index in range(len(sample)):
                    feature_value = sample[feature_index]
                    if feature_value in self.conditional_prob[label][feature_index].keys():
                        label_prob[label] *= self.conditional_prob[label][feature_index][feature_value]
                    else:
                        # label_prob[label] = 0
                        # 拉普拉斯修正
                        label_prob[label] *= 1 / (2*self.num_values[feature_index])
            # 根据概率选择最大的标签
            labels.append(max(label_prob, key=label_prob.get))
        return labels

## 3. 连续属性的朴素贝叶斯分类器

In [4]:
# 定义朴素贝叶斯分类器-连续型
class NaiveBayesClassifierContinuous:
    def __init__(self):
        self.data = None
        self.labels = None
        self.prior_prob = None
        self.conditional_prob = None
        self.num_values = None
        self.mu = None
        self.std = None
    
    def gauss(self, x, mu, std):
        """
        函数说明:
            - 计算高斯函数值
        parameters:
            - mu 均值
            - std 标准差
        return:
            - ans 函数值
        """
        ans = 1/(np.sqrt(2*np.pi)*std)*np.exp(-(x-mu)**2/(2*(std**2)))
        return ans

    # 训练函数
    def fit(self, data, labels):
        """
        函数说明:
            - 计算先验概率和条件概率        
        parameters:
            - data 训练特征矩阵
            - labels 训练标签向量
        return:
            - self
        """
        self.data = data
        self.labels = labels
        # 计算先验概率
        label_counts = Counter(labels)  # 计算每个标签的数量
        num_samples = len(labels)  # 计算样本总数
        self.prior_prob = {label: count / num_samples for label, count in label_counts.items()}  # 计算每个标签的先验概率
        
        # 计算每个标签的第i个特征的均值mu和标准差std
        # 用样本均值和标准差来进行估计
        self.mu = {}; self.std = {}
        for label in label_counts.keys():
            self.mu[label] = {}; self.std[label] = {}
            for feature_index in range(len(data[0])):
                label_indexes = np.where(labels == label)[0]
                feature_values = data[label_indexes, feature_index]
                mu = np.mean(feature_values); std = np.std(feature_values)
                self.mu[label][feature_index] = mu
                self.std[label][feature_index] = std
        
    
        # 计算条件概率
        self.conditional_prob = {}
        self.num_values = {}
        for label in label_counts.keys():
            self.conditional_prob[label] = {}
            for feature_index in range(len(data[0])):
                label_indexes = np.where(labels == label)[0]
                feature_values = data[label_indexes, feature_index]
                self.num_values[feature_index] = len(feature_values)
                value_counts = Counter(feature_values)
                self.conditional_prob[label][feature_index] = {value: self.gauss(value, self.mu[label][feature_index], self.std[label][feature_index]) for value in value_counts.keys()}
        return self
    # 预测函数
    def predict(self, data):
        """
        函数说明:
            - 给定特征值，预测类别
        parameters:
            - data 测试特征向量
        return:
            - self
        """
        labels = []
        for sample in data:
            # 计算每个标签的概率
            label_prob = {label: self.prior_prob[label] for label in self.prior_prob.keys()}
            for label in self.prior_prob.keys():
                for feature_index in range(len(sample)):
                    feature_value = sample[feature_index]
                    # label_prob[label] *= self.conditional_prob[label][feature_index][feature_value]
                    if feature_value in self.conditional_prob[label][feature_index].keys():
                        label_prob[label] *= self.conditional_prob[label][feature_index][feature_value]
                    else:
                        # label_prob[label] = 0
                        # 拉普拉斯修正
                        label_prob[label] *= 1 / (2*self.num_values[feature_index])
            # 根据概率选择最大的标签
            labels.append(max(label_prob, key=label_prob.get))
        return labels

In [5]:
if __name__ == '__main__' :
    fileName = './work/2021冬模式识别数据收集.xlsx'
    data = load_data(fileName)
    X = data.iloc[:, 1:]
    y = data.iloc[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=8)
    X_train = X_train.values; X_test = X_test.values
    y_train = y_train.values; y_test = y_test.values
    NBC = NaiveBayesClassifierContinuous()
    NBD = NaiveBayesClassifierDiscrete()
    NBC.data = X_train; NBC.labels = y_train
    NBD.data = X_train; NBD.labels = y_train
    NBC.fit(X_train, y_train); NBD.fit(X_train, y_train)
    y_pred_C = NBC.predict(X_test)
    y_pred_D = NBD.predict(X_test)
    print('y_pred_C:\t', y_pred_C)
    print('y_pred_D:\t', y_pred_D)
    print('y_test:\t\t', list(y_test))

y_pred_C:	 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0]
y_pred_D:	 [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
y_test:		 [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]


## 4. 比较检验

### 4.1. 二项检验

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=2)

In [7]:
X_train = X_train.values; X_test = X_test.values
y_train = y_train.values; y_test = y_test.values

In [8]:
NBD = NaiveBayesClassifierDiscrete()
NBD.fit(X_train, y_train)
y_pred = NBD.predict(X_test)

In [9]:
errCount = 0
for i in range(len(y_pred)):
    if y_pred[i] != y_test[i]:
        errCount += 1
errRate = errCount/len(y_pred)

In [10]:
errRate

0.09090909090909091

> 假设模型的泛化错误率小于42%，
>
> 即$H_0: \epsilon < 0.42$
>
$$
P(\hat{\epsilon};\epsilon)=C_{m}^n \epsilon^m(1-\epsilon)^n
$$

In [11]:
m = len(y_pred)

In [12]:
import math

In [13]:
P = []
epsilon = 0.42
for n in range(m+1):
    p = math.comb(m, n)*(epsilon**m)*((1-epsilon)**n)
    P.append(p)

In [14]:
sum(P)

0.010989303071931383

In [15]:
# sns.barplot(list(range(m+1)),P)

> 显著性水平$1-\alpha = 0.99$

In [16]:
alpha = 0.01
acc = 0
for i in reversed(list(range(m+1))):
    acc += P[i]
    print(acc)
    if acc > alpha:
        print(i)
        break

1.7924626856583731e-07
3.578744465504131e-06
3.288476340462735e-05
0.00018446761998629912
0.0007071671254403396
0.0019688555868811273
0.004144180520399726
0.006823152113402926
0.00913261038323327
0.01045988525095186
2


In [17]:
errCount

1

> 落在置信区间内
>
> 以99%的置信水平认为，模型分泛化错误率小于42%

### 4.2. t检验 

In [18]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)

In [19]:
NBD = NaiveBayesClassifierDiscrete()
errRate = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    NBD.fit(X_train, y_train)
    y_pred = NBD.predict(X_test)
    errCount = 0
    for i in range(len(y_test)):
        if y_pred[i] != y_test[i]:
            errCount += 1
    errRate.append(errCount/len(y_pred))

In [20]:
errRate

[0.09090909090909091, 0.0, 0.1, 0.2, 0.0]

In [21]:
mu = np.mean(errRate)
sigma = np.var(errRate, ddof = 1) # 修正

In [22]:
mu

0.07818181818181819

> $H_0: \epsilon=0.08$
>
> $\alpha = 0.1$

$$
\tau_t = \frac{\sqrt{k}(\mu -\epsilon_0)}{\sigma}
$$
服从自由度为$k-1$的$t$分布
![](https://jaggar-oss.oss-cn-shanghai.aliyuncs.com/img/427a4d0c7f7fdb96f41962d7289fe29.jpg)

In [23]:
epsilon = 0.08
k = len(y_pred)
t = np.sqrt(k)*(mu - epsilon)/sigma

In [24]:
t

-0.8301922258198599

> 以90%的置信度认为模型的泛化错误率等于8%

### 4.3. 交叉验证t检验

In [25]:
kf = KFold(n_splits=5, shuffle=True, random_state=2)

In [26]:
NBD = NaiveBayesClassifierDiscrete()
NBC = NaiveBayesClassifierContinuous()
errRateD = []; errRateC = []
for train_index, test_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    NBD.fit(X_train, y_train); NBC.fit(X_train, y_train)
    y_pred_D = NBD.predict(X_test); y_pred_C = NBC.predict(X_test)
    errCount_D = 0; errCount_C = 0
    for i in range(len(y_test)):
        if y_pred_D[i] != y_test[i]:
            errCount_D += 1
        if y_pred_C[i] != y_test[i]:
            errCount_C += 1
    errRateD.append(errCount_D/len(y_pred))
    errRateC.append(errCount_C/len(y_pred))

> $H_0$: 两模型性能相当
>
> $\alpha=0.1$

$$
\tau_t=|\frac{\sqrt{k}\mu}{\sigma}|
$$
临界值$t_{\alpha/2,k-1}=2.262$

In [27]:
errRateDelta = np.array(errRateD) - np.array(errRateC)

In [28]:
k = len(y_pred)
mu = errRateDelta.mean()
sigma = errRateDelta.var(ddof=1)

In [29]:
t = np.abs(np.sqrt(k)*mu/sigma)

In [30]:
t

36.14031611621005

> $t>2.262$ 拒绝原假设

In [31]:
np.mean(errRateC)

0.16

In [32]:
np.mean(errRateD)

0.08

> 两学习器有显著的差别 
>
> 离散型平均错误率较小，认为其性能较优

**采用$5\times 2$折交叉验证**

In [33]:
NBD = NaiveBayesClassifierDiscrete()
NBC = NaiveBayesClassifierContinuous()
errRateDD = []; errRateCC = []
for i in range(5):
    errRateD = []; errRateC = []
    kf = KFold(n_splits=2, shuffle=True, random_state=i) # 五次随机
    for train_index, test_index in kf.split(X):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        NBD.fit(X_train, y_train); NBC.fit(X_train, y_train)
        y_pred_D = NBD.predict(X_test); y_pred_C = NBC.predict(X_test)
        errCount_D = 0; errCount_C = 0
        for i in range(len(y_test)):
            if y_pred_D[i] != y_test[i]:
                errCount_D += 1
            if y_pred_C[i] != y_test[i]:
                errCount_C += 1
        errRateD.append(errCount_D/len(y_pred))
        errRateC.append(errCount_C/len(y_pred))
    errRateDD.append(errRateD); errRateCC.append(errRateC)

为了缓解测试错误率的非独立性，仅计算第一次的2折交叉验证的平均值$\mu$，和每次的方差
$$
μ=0.5×(Δ_𝑖^1+Δ_𝑖^2 )\\
σ_𝑖^2=(Δ_𝑖^1−\frac{Δ_𝑖^1+Δ_𝑖^2}{2})^2+(Δ_𝑖^1−\frac{Δ_𝑖^2+Δ_𝑖^2}{2})^2
$$

In [34]:
errRateDelta = np.array(errRateCC) - np.array(errRateDD)

In [35]:
errRateDelta

array([[0.5, 0.2],
       [1.2, 0.4],
       [0.8, 0.6],
       [0.5, 0. ],
       [0.5, 0.1]])

In [36]:
mu = np.mean(errRateDelta[0])
sigma = np.var(errRateDelta, axis=1, ddof=1)

$$
\tau_t=\frac{\mu}{\sqrt{0.2\sum_{i=1}^{5}\sigma_i^2}}
$$
服从自由度为5的$t$分布，临界值$t_{\alpha/2,5}$

$\alpha=0.1$时，$t_{\alpha/2,5}=2.776$

In [37]:
t = mu/np.sqrt(0.2*sum(sigma))

In [38]:
t

1.0188893920442685

> 小于临界值，认为两学习器没有显著差别

### 4.4. McNemar检验

**列联表**
![](https://jaggar-oss.oss-cn-shanghai.aliyuncs.com/img/20221214214255.png)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=2)
X_train, X_test = X.values[train_index], X.values[test_index]
y_train, y_test = y.values[train_index], y.values[test_index]

In [40]:
NBD = NaiveBayesClassifierDiscrete() # 学习器A
NBC = NaiveBayesClassifierContinuous()# 学习器B
errCount_D = 0; errCount_C = 0

NBD.fit(X_train, y_train); NBC.fit(X_train, y_train)
y_pred_D = NBD.predict(X_test); y_pred_C = NBC.predict(X_test)
conTable = np.zeros([2,2])
for i in range(len(y_test)):
    if y_pred_D[i] == 1:
        if y_pred_C[i] == 1:
            conTable[0][0] += 1
        else:
            conTable[1][0] += 1
    else:
        if y_pred_C[i] == 1:
            conTable[0][1] += 1
        else:
            conTable[1][1] += 1

> $H_0: e_{01}=e_{10}$
$$
τ_{χ^2 }=\frac{(𝑏−𝑐)^2}{𝑏+𝑐}
$$
服从自由度为1的卡方分布
>
> $\alpha =0.05$时，临界值为3.8415
>
> $\alpha =0.1$时，临界值为2.7055

In [41]:
chis = (conTable[0][1] - conTable[1][0])**2/(conTable[0][1] + conTable[1][0])

In [42]:
chis

1.0

> 小于临界值，以95%的置信度认为，两学习器的性能没有显著差别

**计算F1-score**
$$
Precision = \frac{TP}{TP + FP}$$$$
Recall = \frac{TP}{TP + FN}$$$$
F-score = \frac{2Precision * Recall}{Precision + Recall}
$$

In [43]:
# 计算混淆矩阵各类别频数
def get_confusion_matrix(y_true, y_pred, flag = 1):
    tn = tp = fn = fp = 0
    for i in range(len(y_true)):
        if y_true[i] == 1:
            if y_pred[i] == 1:
                tp += 1
            else:
                fn += 1
        else:
            if y_pred[i] == 1:
                fp += 1
            else:
                tn += 1
    if flag == 1:
        return tn ,tp, fn, fp
    else:
        return tp, tn, fp, fn


def get_precision(tp, fp):
    if tp == 0:
        return 0.0
    return float(tp) / float(tp + fp)


def get_recall(tp, fn):
    if tp == 0:
        return 0.0
    return float(tp) / float(tp + fn)


def get_fscore(tp, fp, fn):
    precision = get_precision(tp, fp)
    recall = get_recall(tp, fn)
    try:
        return 2 * precision * recall / (precision + recall)
    except:
        return 0.0

In [44]:
def display_f1score(ch = [1, 2, 3, 4], method = NaiveBayesClassifierDiscrete):
    name = ['', '身高', '体重', '脚长', '尺码']
    fileName = './work/2021冬模式识别数据收集.xlsx'
    data = load_data(fileName)
    X = data.iloc[:, ch]
    y = data.iloc[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=8)
    X_train = X_train.values; X_test = X_test.values
    y_train = y_train.values; y_test = y_test.values
    NBD = method()
    NBD.data = X_train; NBD.labels = y_train
    NBD.fit(X_train, y_train)
    y_pred_D = NBD.predict(X_test)
    print('特征为：', end = ' ')
    for item in ch:
        print(name[item], end=' ')
    print()
    tn0, tp0, fn0, fp0 = get_confusion_matrix(y_test, y_pred_D, 0)
    tn1, tp1, fn1, fp1 = get_confusion_matrix(y_test, y_pred_D, 1)
    print('\tf1-score\n0\t%f\n1\t%f'%(get_fscore(tp0, fp0, fn0), get_fscore(tp1, fp1, fn1)))

# 选择特征和分类器
display_f1score([1, 2, 3, 4], NaiveBayesClassifierContinuous)

特征为： 身高 体重 脚长 尺码 
	f1-score
0	0.750000
1	0.333333
